In [ ]:
import torch
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
data_raw = pd.read_csv("../data/winequality-white.processed.csv")
data_train, data_test = train_test_split(data_raw, test_size=0.2)

x_train = data_train.iloc[:, 1 :-1].values
y_train = data_train[["label"]].values

x_test = data_test.iloc[:, 1 :-1].values
y_test = data_test[["label"]].values

In [ ]:
batch_size = 64

dataloader_train = DataLoader(TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float()),
                              shuffle=True, batch_size=batch_size)

dataloader_test = DataLoader(TensorDataset(torch.tensor(x_test).float(), torch.tensor(y_test).float()),
                              shuffle=True, batch_size=batch_size)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
n_bins = 8

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.MLPs = nn.Sequential(
            nn.Linear(n_bins * n_bins + 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x):
        logits = self.MLPs(x)
        return logits
    
model = NeuralNetwork().to(device=device)
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    acc = r2_score(y.cpu(), pred.cpu())
    print(f"Test Error: \n R2 Score: {acc:>8f}, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 500
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader_train, model, loss_fn, optimizer)
    test(dataloader_test, model, loss_fn)
print("Done!")